In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import mode
from scipy.stats.distributions import randint, uniform, expon
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [2]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [3]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived'].ravel()

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
def write_test(df, mod, n=1):
    dft = pd.DataFrame(np.c_[df['PassengerId'],mod.predict(df)], columns=['PassengerId', 'Survived'])
    dft.to_csv('Predictions_Teste{}.csv'.format(n), index=False)

In [47]:
class Impute_Better(BaseEstimator, TransformerMixin):
    """ Impute missing data in a more effective way, groupwise by Sex and Pclass"""
    def __init__(self, col='Age', func=np.mean, mode=2):
        self.col = col
        self.func = func
        self.mode = mode
    def fit(self, X, y=None):
        if self.mode == 2:
            self.vals = X.groupby(by=['Sex', 'Pclass'])[self.col].agg(self.func)
        elif self.mode == 1:
            self.vals = X.groupby(by=['Sex'])[self.col].agg(self.func)
        else:
            self.vals = X[self.col].agg(self.func)
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        idx_na = X.loc[:,self.col].isna()
        if self.mode == 2:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i][0]) & (X['Pclass'] == self.vals.index[i][1]) &
                       idx_na),self.col] = self.vals[i]
        elif self.mode == 1:
            for i in range(len(self.vals)):
                Xt.loc[((X['Sex'] == self.vals.index[i]) & idx_na),self.col] = self.vals[i]
        else:
            Xt.loc[idx_na, self.col] = self.vals
        return Xt

In [48]:
class New_Features(BaseEstimator, TransformerMixin):
    """ Add new features, in this case Parch + SibSp """
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        Xt = X.copy()
        Xt['Nrelatives'] = Xt['Parch'] + Xt['SibSp']
        return Xt

In [9]:
class Transf_Conti(BaseEstimator, TransformerMixin):
    """ Transforms a feature in categorical or startardizes it"""
    def __init__(self, n=4, q=False, conti=False):
        self.n = n
        self.q = q
        self.conti=conti
    def fit(self, X, y=None):
        if not self.conti:
            if not self.q:
                self.binst = pd.cut(X.values[:,0], bins=self.n, retbins=True, duplicates='drop')[1]
            if self.q:
                self.binst = pd.qcut(X.values[:,0], q=self.n, retbins=True, duplicates='drop')[1]
            self.higher = self.binst[-1]
            self.lower = self.binst[0]
        else:
            self.scaler_transf = StandardScaler()
            self.scaler_transf.fit(X)
        
        return self
    def transform(self, X, y=None):
        if not self.conti:
            val_ret = pd.cut(X.values[:,0], bins=self.binst, labels=False, include_lowest=True, duplicates='drop')
            val_ret[X.values[:,0] > self.higher] = self.n - 1
            val_ret[X.values[:,0] < self.lower] = 0
            return val_ret.reshape((-1, 1))
        else:
            return self.scaler_transf.transform(X)

In [10]:
class Select_Features(BaseEstimator, TransformerMixin):
    def __init__(self, select=False):
        self.select=select
    def fit(self, X, y=None):
        return self
    def transform(self, X,y=None):
        if not self.select:
            return X
        return X[:, self.select]

In [11]:
Pipe_Impute = Pipeline([
    #Impute Pipeline, the one which imputes missing data and creates new features
    ('Age_Impute', Impute_Better(col='Age', func=np.median)),
    ('Fare_Impute', Impute_Better(col='Fare', func=np.median)),
    ('Embarked_Impute', Impute_Better(col='Embarked', func=lambda x: mode(x)[0][0])),
    ('New_Features', New_Features())
])

In [12]:
Pipe_Preprocess = ColumnTransformer([
    ### Preprocessing pipeline. Encodes, standardizes
    ('Ordinal_Encode', OrdinalEncoder(), ['Sex','Pclass']), 
    ('Age_Encode', Transf_Conti(), ['Age']), 
    ('Fare_Encode', Transf_Conti(), ['Fare']), 
    ('Keep', 'passthrough', ['Parch', 'SibSp', 'Nrelatives']), 
    ('Embarked_Encode', OneHotEncoder(), ['Embarked']),
])

# Random Forest

In [20]:
FullPipePredict_RF = Pipeline([
    ('Impute_Pipeline', Pipe_Impute),
    ('Preprocess_Pipeline', Pipe_Preprocess),
    ('Feature_Selection', Select_Features()),
    ('Predict', RandomForestClassifier(n_jobs=-1))
])

In [32]:
param_RF = {
    'Preprocess_Pipeline__Age_Encode__conti':[1],
    'Preprocess_Pipeline__Fare_Encode__conti':[1],
    'Feature_Selection__select':[[0,1,2,3,6]],
    'Predict__n_estimators':randint(120, 180),
    'Predict__max_leaf_nodes':randint(20,32),
}

In [33]:
grid_RF = RandomizedSearchCV(FullPipePredict_RF, param_distributions=param_RF, cv=10, n_iter=60,
                            scoring='accuracy', verbose=10)

In [34]:
grid_RF.fit(X_train, y_train)

Fitting 10 folds for each of 60 candidates, totalling 600 fits
[CV 1/10; 1/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=136, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 1/10; 1/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=136, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.789 total time=   0.1s
[CV 2/10; 1/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=136, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 2/10; 1/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=136, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.865 total time=   0.1s
[CV 3/10; 1/60] START Feature_Selection__select=[0, 1, 2, 3

[CV 10/10; 2/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=171, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.831 total time=   0.2s
[CV 1/10; 3/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=127, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 1/10; 3/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=127, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.744 total time=   0.1s
[CV 2/10; 3/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=127, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 2/10; 3/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=127, Pre

[CV 9/10; 4/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=154, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.888 total time=   0.1s
[CV 10/10; 4/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=154, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 10/10; 4/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=154, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.820 total time=   0.1s
[CV 1/10; 5/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=31, Predict__n_estimators=178, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 1/10; 5/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=31, Predict__n_estimators=178, Pr

[CV 8/10; 6/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=147, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.798 total time=   0.1s
[CV 9/10; 6/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=147, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 9/10; 6/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=147, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.876 total time=   0.1s
[CV 10/10; 6/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=147, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 10/10; 6/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=147, Pr

[CV 7/10; 8/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=24, Predict__n_estimators=145, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.843 total time=   0.1s
[CV 8/10; 8/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=24, Predict__n_estimators=145, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 8/10; 8/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=24, Predict__n_estimators=145, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.809 total time=   0.2s
[CV 9/10; 8/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=24, Predict__n_estimators=145, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 9/10; 8/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=24, Predict__n_estimators=145, Prep

...

[CV 3/10; 54/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=153, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.753 total time=   0.3s
[CV 4/10; 54/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=153, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 4/10; 54/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=153, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.899 total time=   0.3s
[CV 5/10; 54/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=153, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 5/10; 54/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=23, Predict__n_estimators=153,

[CV 2/10; 56/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=142, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.831 total time=   0.2s
[CV 3/10; 56/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=142, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 3/10; 56/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=142, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.753 total time=   0.2s
[CV 4/10; 56/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=142, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 4/10; 56/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=30, Predict__n_estimators=142,

[CV 1/10; 58/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=144, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.767 total time=   0.2s
[CV 2/10; 58/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=144, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 2/10; 58/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=144, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.854 total time=   0.2s
[CV 3/10; 58/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=144, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 3/10; 58/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=27, Predict__n_estimators=144,

[CV 10/10; 59/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=25, Predict__n_estimators=151, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.820 total time=   0.2s
[CV 1/10; 60/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=131, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 1/10; 60/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=131, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1;, score=0.778 total time=   0.2s
[CV 2/10; 60/60] START Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=131, Preprocess_Pipeline__Age_Encode__conti=1, Preprocess_Pipeline__Fare_Encode__conti=1
[CV 2/10; 60/60] END Feature_Selection__select=[0, 1, 2, 3, 6], Predict__max_leaf_nodes=28, Predict__n_estimators=131

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('Impute_Pipeline',
                                              Pipeline(steps=[('Age_Impute',
                                                               Impute_Better(func=<function median at 0x000001448ABE18B0>)),
                                                              ('Fare_Impute',
                                                               Impute_Better(col='Fare',
                                                                             func=<function median at 0x000001448ABE18B0>)),
                                                              ('Embarked_Impute',
                                                               Impute_Better(col='Embarked',
                                                                             func=<function <lambda> at 0x00000144900E5F70>)),
                                                              ('New_Features...
                   param_distribu

In [45]:
grid_RF.best_score_

0.8440699126092385

In [46]:
grid_RF.best_params_

{'Feature_Selection__select': [0, 1, 2, 3, 6],
 'Predict__max_leaf_nodes': 27,
 'Predict__n_estimators': 139,
 'Preprocess_Pipeline__Age_Encode__conti': 1,
 'Preprocess_Pipeline__Fare_Encode__conti': 1}

In [55]:
grid_RF.best_estimator_[-1].oob_score_

0.8327721661054994

In [53]:
best_RF = grid_RF.best_estimator_

In [555]:
best_RF[-1].feature_importances_

array([0.45374799, 0.13817604, 0.14346081, 0.18145855, 0.08315661])

In [54]:
write_test(test, best_RF, '_RF_Well_Tuned2')

In [556]:
joblib.dump(best_RF[-1], 'BestRFsofar.pkl')

['BestRFsofar.pkl']